In [1]:
using Pkg; Pkg.activate("/home/dhairyagandhi96/modelzoo/model-zoo/script/../vision/mnist"); Pkg.instantiate(); Pkg.add(PackageSpec(name="CUDAdrv", rev = "master")); Pkg.add(PackageSpec(name="CUDAnative", rev = "master")); Pkg.add(PackageSpec(name="CuArrays", rev = "master")); Pkg.add(PackageSpec(name="Flux", rev = "master"))

using Flux, Flux.Data.MNIST, Statistics
using Flux: throttle, params
using Juno: @progress

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/JuliaGPU/CUDAdrv.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/vision/mnist/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CUDAnative.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/vision/mnist/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CuArrays.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/vision/mnist/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/FluxML/Flux.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Pr

Extend distributions slightly to have a numerically stable logpdf for `p` close to 1 or 0.

In [2]:
using Distributions
import Distributions: logpdf
logpdf(b::Bernoulli, y::Bool) = y * log(b.p + eps(Float32)) + (1f0 - y) * log(1 - b.p + eps(Float32))

logpdf (generic function with 62 methods)

Load data, binarise it, and partition into mini-batches of M.

In [3]:
X = float.(hcat(vec.(MNIST.images())...)) .> 0.5
N, M = size(X, 2), 100
data = [X[:,i] for i in Iterators.partition(1:N,M)]


################################# Define Model #################################

600-element Array{BitArray{2},1}:
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false

Latent dimensionality, # hidden units.

In [4]:
Dz, Dh = 5, 500

(5, 500)

Components of recognition model / "encoder" MLP.

In [5]:
A, μ, logσ = Dense(28^2, Dh, tanh), Dense(Dh, Dz), Dense(Dh, Dz)
g(X) = (h = A(X); (μ(h), logσ(h)))
z(μ, logσ) = μ + exp(logσ) * randn(Float32)

z (generic function with 1 method)

Generative model / "decoder" MLP.

In [6]:
f = Chain(Dense(Dz, Dh, tanh), Dense(Dh, 28^2, σ))


####################### Define ways of doing things with the model. #######################

Chain(Dense(5, 500, tanh), Dense(500, 784, NNlib.σ))

KL-divergence between approximation posterior and N(0, 1) prior.

In [7]:
kl_q_p(μ, logσ) = 0.5f0 * sum(exp.(2f0 .* logσ) + μ.^2 .- 1f0 .+ logσ.^2)

kl_q_p (generic function with 1 method)

logp(x|z) - conditional probability of data given latents.

In [8]:
logp_x_z(x, z) = sum(logpdf.(Bernoulli.(f(z)), x))

logp_x_z (generic function with 1 method)

Monte Carlo estimator of mean ELBO using M samples.

In [9]:
L̄(X) = ((μ̂, logσ̂) = g(X); (logp_x_z(X, z.(μ̂, logσ̂)) - kl_q_p(μ̂, logσ̂)) * 1 // M)

loss(X) = -L̄(X) + 0.01f0 * sum(x->sum(x.^2), params(f))

loss (generic function with 1 method)

Sample from the learned model.

In [10]:
modelsample() = rand.(Bernoulli.(f(z.(zeros(Dz), zeros(Dz)))))


################################# Learn Parameters ##############################

evalcb = throttle(() -> @show(-L̄(X[:, rand(1:N, M)])), 30)
opt = ADAM(params(A, μ, logσ, f))
@progress for i = 1:20
  @info "Epoch $i"
  Flux.train!(loss, zip(data), opt, cb=evalcb)
end


################################# Sample Output ##############################

#using Images

#img(x) = Gray.(reshape(x, 28, 28))

#cd(@__DIR__)
#sample = hcat(img.([modelsample() for i = 1:10])...)
#save("sample.png", sample)

┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Info: Epoch 1
└ @ Main.##360 string:9
┌ Warning: train!(loss, data, opt) is deprecated; use train!(loss, params, data, opt) instead
│   caller = ip:0x0
└ @ Core :-1
-(L̄(X[:, rand(1:N, M)])) = 542.5721f0 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 180.3149f0 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 180.8608f0 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 181.59308f0 (tracked)
┌ Info: Epoch 2
└ @ Main.##360 string:9
-(L̄(X[:, rand(1:N, M)])) = 169.0182f0 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 165.75212f0 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 162.6349f0 (tracked)
┌ Info: Epoch 3
└ @ Main.##360 string:9
-(L̄(X[:, rand(1:N, M)])) = 154.15788f0 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 158.55838f0 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 155.69676f0 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 157.82607f0 (tracked)
┌ Info: Epoch 4
└ @ Main.##360 string:9
-(L̄(X[:, rand(1:N, M)])) = 143.8975f0 (track